## Survival Analysis: time varying survival regression

Materials:
- Time varying survival regression [Link](https://lifelines.readthedocs.io/en/latest/Time%20varying%20survival%20regression.html)
- TA Review on Accelerated Failure Time Models [Link](https://www.ripublication.com/ijss17/ijssv12n2_15.pdf)
- The Cox Proportional Hazards Model [Link](https://bookdown.org/sestelo/sa_financial/cox.html)
- On Ranking in Survival Analysis: Bounds on the Concordance Index [Link](https://proceedings.neurips.cc/paper/2007/file/33e8075e9970de0cfea955afd4644bb2-Paper.pdf)

**Dataset creation for time-varying regression** <br/>
lifelines requires that the dataset be in what is called the long format. This looks like one row per state change, including an ID, the left (exclusive) time point, and right (inclusive) time point.

In [42]:
import pandas as pd
from lifelines.utils import to_long_format

In [43]:
# data from relational database with durations and event flags
base_df = pd.DataFrame([
  {'id': 1, 'duration': 10, 'event': True, 'var1': 0.1},
  {'id': 2, 'duration': 12, 'event': True, 'var1': 0.5}
])


In [44]:
base_df

,id,duration,event,var1
0,1,10,True,0.1
1,2,12,True,0.5


In [45]:
base_df = to_long_format(base_df, duration_col="duration")

In [46]:
base_df

,id,event,var1,start,stop
0,1,True,0.1,0,10
1,2,True,0.5,0,12


In [47]:
from lifelines.utils import add_covariate_to_timeline

In [48]:
# secondary dataset that references future measurements - variables that change over time
# time - duration from the entry point
cv = pd.DataFrame([
  {'id': 1, 'time': 0, 'var2': 1.4},
  {'id': 1, 'time': 4, 'var2': 1.2},
  {'id': 1, 'time': 8, 'var2': 1.5},
  {'id': 2, 'time': 0, 'var2': 1.6},

])

In [49]:
cv

,id,time,var2
0,1,0,1.4
1,1,4,1.2
2,1,8,1.5
3,2,0,1.6


In [52]:
# use `add_covariate_to_timeline` to fold the covariate dataset into original dataset
df = add_covariate_to_timeline(base_df, cv, 
                               duration_col="time", 
                               id_col="id", 
                               event_col="event").columns()

In [54]:
cols = ['id','start','stop','var1','var2','event']

In [56]:
df[cols]

,id,start,stop,var1,var2,event
0,1,0,4.0,0.1,1.4,False
1,1,4,8.0,0.1,1.2,False
2,1,8,10.0,0.1,1.5,True
3,2,0,12.0,0.5,1.6,True


In [ ]:
# we can add multiple covariates with pipe:

# df = base_df.pipe(add_covariate_to_timeline, cv1, duration_col="time", id_col="id", event_col="event")\
#             .pipe(add_covariate_to_timeline, cv2, duration_col="time", id_col="id", event_col="event")\
#             .pipe(add_covariate_to_timeline, cv3, duration_col="time", id_col="id", event_col="event")

In [58]:
# secondary dataset - event-based with values in the matrix denote times since the subject's birth

event_df = pd.DataFrame([
    {'id': 1, 'E1': 1.0},
    {'id': 2, 'E1': None},
    {'id': 3, 'E1': 3.0},
])
event_df

,id,E1
0,1,1.0
1,2,NaN
2,3,3.0


In [ ]:
# cannot add to baseline df, but can convert and add with `covariates_from_event_matrix`

In [59]:
from lifelines.utils import covariates_from_event_matrix

In [60]:
cv = covariates_from_event_matrix(event_df, id_col="id")

In [61]:
cv

,id,duration,E1
0,1,1.0,1
1,2,inf,1
2,3,3.0,1


In [63]:
base_df = add_covariate_to_timeline(base_df, cv, duration_col="duration", id_col="id", event_col="event")


In [64]:
base_df

,start,E1,var1,stop,id,event
0,0.0,NaN,0.1,1.0,1,False
1,1.0,1.0,0.1,10.0,1,True
2,0.0,NaN,0.5,12.0,2,True
